In [1]:
# Group Members
# - Member 1: [Muhamad Hafizuddin bin Othman], [SW01081170]
# - Member 2: [Naufal bin Nujaimin], [SW01081171]

In [2]:
# Import the necessary libraries
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from gensim import corpora
from gensim.models import LdaModel
from gensim.models.coherencemodel import CoherenceModel

# Download NLTK resources
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
# Read the data
df = pd.read_csv('news_dataset.csv')
documents = df['text'].dropna().tolist()  # Use only the 'text' column and remove null values

In [4]:
# Perform text pre-processing
stop_words = set(stopwords.words('english'))  # Create a set of English stopwords
lemmatizer = WordNetLemmatizer()  # Initialize a WordNet lemmatizer

def preprocess_text(text):
    tokens = word_tokenize(text.lower())  # Tokenize the text into words and convert to lowercase
    tokens = [token for token in tokens if token.isalnum()]  # Filter out non-alphanumeric tokens
    tokens = [token for token in tokens if token not in stop_words]  # Remove stopwords from the tokens
    tokens = [lemmatizer.lemmatize(token) for token in tokens]  # Lemmatize each token
    return tokens  # Return the preprocessed tokens

preprocessed_documents = [preprocess_text(doc) for doc in documents]  # Preprocess each document in the list

In [5]:
# Create document-term matrix
dictionary = corpora.Dictionary(preprocessed_documents)  # Create a Gensim Dictionary object from the preprocessed documents
dictionary.filter_extremes(no_below=15, no_above=0.5)  # Filter out tokens that appear in less than 15 documents or more than 50% of the documents
corpus = [dictionary.doc2bow(doc) for doc in preprocessed_documents]  # Convert each preprocessed document into a bag-of-words representation using the dictionary

In [6]:
# Perform LDA using Gensim
lda_model = LdaModel(corpus, num_topics=4, id2word=dictionary, passes=15)  # Train an LDA model on the corpus with 4 topics using Gensim's LdaModel class

In [7]:
# Evaluate the LDA model using Coherence score
coherence_model_lda = CoherenceModel(model=lda_model, texts=preprocessed_documents, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()

In [8]:
# Display the coherence score
print(f'Topic Coherence Score (C_V): {coherence_lda:.4f}')

Topic Coherence Score (C_V): 0.6241


In [9]:
# Interpret the result
article_labels = []  # Empty list to store dominant topic labels for each document

# Iterate over each processed document
for i, doc in enumerate(preprocessed_documents):
    bow = dictionary.doc2bow(doc)  # For each document, convert to BOW representation
    topics = lda_model.get_document_topics(bow)  # Get list of topic probabilities
    dominant_topic = max(topics, key=lambda x: x[1])[0]  # Determine topic with highest probability
    article_labels.append(dominant_topic)  # Append to the list

# Create DataFrame
df_result = pd.DataFrame({"Article": documents, "Topic": article_labels})

# Print the DataFrame
print("Table with Articles and Topic:")
print(df_result)
print()

Table with Articles and Topic:
                                                 Article  Topic
0      I was wondering if anyone out there could enli...      3
1      I recently posted an article asking what kind ...      3
2      \nIt depends on your priorities.  A lot of peo...      3
3      an excellent automatic can be found in the sub...      3
4      : Ford and his automobile.  I need information...      3
...                                                  ...    ...
11091  Secrecy in Clipper Chip\n\nThe serial number o...      2
11092  Hi !\n\nI am interested in the source of FEAL ...      2
11093  The actual algorithm is classified, however, t...      2
11094  \n\tThis appears to be generic calling upon th...      1
11095  \nProbably keep quiet and take it, lest they g...      3

[11096 rows x 2 columns]



In [10]:
# Print the top terms for each topic
print("Top Terms for Each Topic:")
for topic_id in range(lda_model.num_topics):
    print(f"Top terms for Topic #{topic_id}:")
    top_terms = lda_model.show_topic(topic_id, topn=10)
    print([term[0] for term in top_terms])
    print()

Top Terms for Each Topic:
Top terms for Topic #0:
['1', 'x', '0', 'max', '2', 'q', '7', 'g', 'r', '3']

Top terms for Topic #1:
['would', 'people', 'government', 'one', 'think', 'law', 'right', 'year', 'q', 'president']

Top terms for Topic #2:
['key', 'use', 'file', 'system', 'one', 'chip', 'get', 'program', 'window', 'also']

Top terms for Topic #3:
['one', 'would', 'people', 'god', 'like', 'time', 'know', 'say', 'get', 'think']



In [11]:
# Print the top terms for each topic with weight
print("Top Terms for Each Topic with Weight:")
for idx, topic in lda_model.print_topics():
    print(f"Topic {idx}:")
    terms = [term.strip() for term in topic.split("+")]
    for term in terms:
        weight, word = term.split("*")
        print(f"- {word.strip()} (weight: {weight.strip()})")
    print()

Top Terms for Each Topic with Weight:
Topic 0:
- "1" (weight: 0.055)
- "x" (weight: 0.047)
- "0" (weight: 0.041)
- "max" (weight: 0.039)
- "2" (weight: 0.038)
- "q" (weight: 0.026)
- "7" (weight: 0.026)
- "g" (weight: 0.025)
- "r" (weight: 0.025)
- "3" (weight: 0.024)

Topic 1:
- "would" (weight: 0.010)
- "people" (weight: 0.007)
- "government" (weight: 0.006)
- "one" (weight: 0.006)
- "think" (weight: 0.006)
- "law" (weight: 0.005)
- "right" (weight: 0.005)
- "year" (weight: 0.005)
- "q" (weight: 0.005)
- "president" (weight: 0.004)

Topic 2:
- "key" (weight: 0.010)
- "use" (weight: 0.009)
- "file" (weight: 0.008)
- "system" (weight: 0.007)
- "one" (weight: 0.006)
- "chip" (weight: 0.006)
- "get" (weight: 0.005)
- "program" (weight: 0.005)
- "window" (weight: 0.005)
- "also" (weight: 0.005)

Topic 3:
- "one" (weight: 0.010)
- "would" (weight: 0.009)
- "people" (weight: 0.008)
- "god" (weight: 0.006)
- "like" (weight: 0.006)
- "time" (weight: 0.006)
- "know" (weight: 0.006)
- "say" (we

In [12]:
# Coherence Score Interpretation:
# The coherence score indicates how interpretable and meaningful the topics generated by the LDA model are.
# A higher coherence score suggests the topics are more coherent and well-defined, making them easier to interpret and analyze.
# A lower coherence score may indicate that the topics are less coherent and may require further refinement.